<a href="https://colab.research.google.com/github/ahmedchafiq/encg/blob/main/public_Data_Mixing_0_92776.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
anthonytherrien_predicting_loan_payback_vault_path = kagglehub.dataset_download('anthonytherrien/predicting-loan-payback-vault')

print('Data source import complete.')


100%|██████████| 4.35M/4.35M [00:01<00:00, 3.48MB/s]

Extracting files...


Data source import complete.


In [2]:
import pandas as pd

In [3]:
# Define the blending function
def blend_submissions(weight_dict, output_path):
    # Initialize list to store loaded DataFrames
    dataframes = []

    # Load each submission with its weight
    for path, weight in weight_dict.items():
        # Read the CSV file
        df = pd.read_csv(path)

        # Add a weighted prediction column
        df["weighted_pred"] = df["loan_paid_back"] * weight

        # Append to list
        dataframes.append(df[["id", "weighted_pred"]])

    # Merge all submissions on 'id'
    merged = dataframes[0]
    for df in dataframes[1:]:
        # Merge on id
        merged = merged.merge(df, on="id", how="inner", suffixes=("", "_dup"))

        # Combine duplicate weighted_pred columns if any
        if "weighted_pred_dup" in merged.columns:
            merged["weighted_pred"] += merged["weighted_pred_dup"]
            merged.drop(columns=["weighted_pred_dup"], inplace=True)

    # Compute total weight
    total_weight = sum(weight_dict.values())

    # Compute blended prediction
    merged["loan_paid_back"] = merged["weighted_pred"] / total_weight

    # Prepare final DataFrame
    blended = merged[["id", "loan_paid_back"]]

    # Save blended submission
    blended.to_csv(output_path, index=False)

    # Print confirmation
    print(f"✅ Blended submission saved to {output_path}")

In [4]:
# Define the main function
def main():
    # Define file paths and their respective weights
    weight_dict = {
        "/kaggle/input/predicting-loan-payback-vault/submission.csv": 2.2,
        "/kaggle/input/predicting-loan-payback-vault/submission (1).csv": 0.1,
    }

    # Call blend function
    blend_submissions(weight_dict, output_path="submission.csv")

In [9]:
import pandas as pd
from google.colab import files

def main():
    print("📥 Upload ton fichier CSV…")
    uploaded = files.upload()

    # Récupère automatiquement le nom du fichier
    csv_file = list(uploaded.keys())[0]

    print(f"\n➡️ Fichier détecté : {csv_file}")

    # Lecture sécurisée
    try:
        df = pd.read_csv(csv_file)
        print("\n✅ Fichier chargé avec succès !")
        print(df.head())
    except Exception as e:
        print("\n❌ Erreur lors du chargement :")
        print(e)

# Lancer automatiquement
if __name__ == "__main__":
    main()


📥 Upload ton fichier CSV…


Saving archive (1).zip to archive (1).zip

➡️ Fichier détecté : archive (1).zip

❌ Erreur lors du chargement :
Multiple files found in ZIP file. Only one file per ZIP: ['submission (1).csv', 'submission.csv']
